Homework 2 Qn1 

In [2]:
import random
import numpy as np

g = random.Random(1234)

# generate an Mt/G/1 queue
# arrival rate lambda(t)=5+cos(2*pi*t) and service time distribution F(x)=1-exp(-4*sqrt(x))
# output: max_{0<=t<=6} Q(t) > 5, i.e. wheter there are 5 or more jobs in the queue on the interval [0,6] 
def MtG1_queue():
    #system parameter
    lm_bar=6    
    
    t = 0 # initialize the clock to zero
    X = 0 # initialize the number of people in the system to zero
    I = 0 # indicator about whether Q>5, Q=(X-1)^+ 
    
    # generate the first arrival time
    A = 0 # arrival time of the time-homogeneous Poisson process (upper bound process)
    while True:
        A = A - np.log(g.random())/lm_bar
        V = g.random()
        if V <= (5 + np.cos(2*np.pi*(A))) / lm_bar:
            break
    
    D = float("inf"); # set the next departure time as infinity when there is no one in service
    
    if A<D:
        t = A
        E = 1 # next event is an arrival
    else:
        t = D
        E = 2 # next event is a depature
    
    while t < 6: # 6 is the simulation time horizon
        if E == 1: # next event is an arrival
            X = X + 1 # update the state of the system
            if X >= 6: # there are more than 5 jobs in queue
                I = 1
                break
            if X == 1: # there is no job in service before this arrival
                D = t + (-np.log(g.random()) / 5)**2
            #update the next arrival time
            while True:
                A = A - np.log(g.random())/lm_bar
                V = g.random()
                if V <= (5 + np.cos(2*np.pi*(A))) / lm_bar:
                    break
        else: # next event is a departure
            X = X - 1 # update the state of the system
            if X > 0: # there is still job in the system
                D = t + (-np.log(g.random()) / 5)**2
            else: # there is no job in the system
                D = float("inf")
        # update the clock time to the next event time
        if A<D:
            t = A
            E = 1 # next event is an arrival
        else:
            t = D
            E = 2 # next event is a depature
    
    return I

# Esimate P(max_{0<=t<=6} Q(t) > 5) using the sample average of n iid replications    
n = 10000 # number of replications
I = n * [0] # pre-assign the space to I
for i in range(n):
    I[i] = MtG1_queue()

print(np.mean(I)) # output the sample average

0.3358


Homework 2 Qn2

In [5]:
import random
import numpy as np

g = random.Random(1234)

# generate normal random variables
def Normal_rv():
    H = -2 * np.log(g.random())
    Theta = 2 * np.pi * g.random()
    Z1 = np.sqrt(H) * np.cos(Theta)
    Z2 = np.sqrt(H) * np.sin(Theta)
    return Z1

# generate one copy of C_T=(1/4*int_{0}^{4}S(t)dt - 42)^+ 
# S(t)= S(0)*exp(0.15*t+0.02*B(t)) where B(t) is a standard Brownian Motion
# Estimate the integral using sum_{i=0}^{k-1}S(4*i/k)*4/k
def Asian_Call_Option():
    # system parameters
    mu = 0.15
    sigma = 0.02
    T = 4
    K = 42 
    
    k = 100 # discretization grid numbers
    delta = T/k
    
    S = 40 # initialize S to S(0)
    In = S*delta
    
    for i in range(k-1):
        S = S * np.exp( mu * delta + sigma * np.sqrt(delta) * Normal_rv())
        In += S*delta
        
    CT = max( In/4 - K, 0)
    return CT

# esimate E[C_T] using the sample average of n iid replications
n = 10000
CT = n * [0] # pre-assign the space to I
for i in range(n):
    CT[i] = Asian_Call_Option()

print(np.mean(CT)) # output the sample average

12.6769091031


Homework 2 Qn3

In [6]:
import random
import numpy as np

g = random.Random(1234)

# Generate a Geometric random variable with probability of success p
def Geometric(p):
    X = 1
    f = p
    F = f
    U = g.random()
    while U > F:
        X += 1
        f = f * (1-p)
        F += f
    return X

# Generate one copy of R(T)
def Inventory():
    # system parameters
    c = 0.2
    r = 1
    lm = 1
    mu = 2
    p = 0.2
    s = 10
    S = 50
    T = 100
    
    # initialization
    t = 0
    I = S
    R = 0
    Y = 0
    
    # generate the next event time
    A = - np.log(g.random()) / lm # next purchase time
    D = float("inf") # next delivery time which is set to infinity when there is no order to be delivered
    
    # move the clock time to the next event time
    tp = t
    if A < D: # next event is a purchace
        t = A
        E = 1
    else: # next event is a delivery
        t = D
        E = 2
    
    while t < T:
        R = R - c * I * (t - tp)
        if E == 1: # next event is a purchace
            B = Geometric (p)
            m = min(B,I)
            R += r * m
            I = I - m
            if I < s and Y == 0: # whether we need to order more product
                Y = S - I
                D = t - np.log(g.random()) / mu  # update the next deliver time
            A = t - np.log(g.random()) / lm # update the next purchase time
        else: # next event is a delivery
            I = I + Y
            Y = 0
            D = float("inf") # update the next deliver time
        # move the clock time to the next event time
        tp = t
        if A < D: # next event is a purchace
            t = A
            E = 1
        else: # next event is a delivery
            t = D
            E = 2
            
    R = R - c * I * (T - tp)
    return R  
    
# esimate E[R(T)] using the sample average of n iid replications
n = 10000
R = n * [0] # pre-assign the space to I
for i in range(n):
    R[i] = Inventory()

print(np.mean(R)) # output the sample average

-121.778370476


Homework 2 Qn4

In [7]:
import random
import numpy as np

g = random.Random(1234)

# generate an M/M/1 Processor Sharing queue
# estimate the long run average amount of time a customer spends in the system
def MM1_PS(T): # we estimate the long run using (1/N(T))*sum_{i=1}^{N(T)}W_i where N(T) is total number of arrival before T
    # system parameters
    lm = 0.9
    mu = 1
    
    # initialization
    t = 0 
    En = [] # En records the time job enters the system
    R = [] # R records the remaining service time of jobs currently in service
    W = 0 # W records the total waiting time of customers
    NT = 0 # NT records the number of arrivals before time T
    
    # generate the next event time
    A = - np.log(g.random()) / lm # next arrival time
    D = float("inf") # next departure time. When the system is empty we set D = infty
    
    # set the clock time to the next event time
    tp = t
    if A < D: # next event is an arrival
        t = A
        E = 1
    else: # next event is a depature
        t = D
        E = 2
    
    while t < T:
        Q = len(R)
        if Q > 0: 
            for i in range(Q):
                R[i] = R[i] - (t - tp) / Q # update the remaining service time
            
        if E == 1: # next event is an arrival
            V = - np.log(g.random()) / mu
            En.append(t)
            R.append(V)
            
            # update the next event time
            A = A - np.log(g.random()) / lm
            Q = len(R)
            D = t + min(R) * Q
            
        else: # next event is a departure 
            # find the position of the departing customer
            Rm = min(R)
            Ind = R.index(Rm) 
            Ar = En[Ind] # find the arrival time of the departing customer
            W += t - Ar # record the waiting time of the departing customer
            NT += 1 
            
            # remove the depearting cusotmer
            En.remove(Ar) 
            R.remove(Rm)
            
            # update the next event time
            Q = len(R)
            if Q == 0:
                D = float("inf")
            else:
                D = t + min(R) * Q
        
                
        # set the clock time to the next event time
        tp = t
        if A < D: # next event is an arrival
            t = A
            E = 1
        else: # next event is a depature
            t = D
            E = 2
    
    # to avoid observation bias, we have to generate the system until all the customers that are in the system at time T depart
    QT = len(R) 
    while QT > 0:
        Q = len(R)
        if Q > 0: # update R
            for i in range(Q):
                R[i] = R[i] - (t - tp) / Q
            
        if E == 1: # next event is an arrival
            V = - np.log(g.random()) / mu
            En.append(t)
            R.append(V)
            
            # update the next event time
            A = A - np.log(g.random()) / lm
            D = t + min(R) * Q
            
        else: # next event is a departure 
            # find the position of the departing customer
            Rm = min(R)
            Ind = R.index(Rm) 
            Ar = En[Ind] # find the arrival time of the departing customer
            
            if Ar < T:  
                W += t - Ar # record the waiting time of the departing customer
                NT += 1
                QT = QT -1
            
            # remove the depearting cusotmer
            En.remove(Ar) 
            R.remove(Rm)
            
            # update the next event time
            Q = len(R)
            if Q == 0:
                D = float("inf")
            else:
                D = t + min(R) * Q
        
                
        # set the clock time to the next event time
        tp = t
        if A < D: # next event is an arrival
            t = A
            E = 1
        else: # next event is a depature
            t = D
            E = 2
    
    EW = W / NT
    return EW

print(MM1_PS(10000))

7.66476473864
